In [39]:
import pandas as pd
df = pd.read_csv("anr-volatility.csv")
#df = df[(df.last_price < 500)]

In [40]:
df

,sec_idx,security,volatility,volatility_state,symbol,last_price,mu,sigma,iv,strike,option_price,option,position
0,1,AA US Equity,0.023830,0,AA,28.2500,0.357587,0.330416,0.42958,30.0,4.40,AA Jan 17 2020 30 Call,1
1,3,AAL US Equity,0.015531,0,AAL,34.9700,0.240695,0.188852,0.36023,36.0,3.20,AAL Aug 16 2019 36 Call,1
2,4,AAOI US Equity,NaN,0,AAOI,15.1500,0.250936,0.573151,0.69299,17.5,4.13,AAOI Sep 20 2019 17.5 Call,1
3,8,ACB US Equity,0.063592,0,ACB,7.0600,0.198110,0.498128,0.67684,8.0,1.12,ACB Sep 20 2019 8 Call,1
4,12,ADM US Equity,0.012726,0,ADM,41.7500,0.200858,0.105393,0.21054,42.0,2.61,ADM Sep 20 2019 42 Call,1
5,15,AEM US Equity,0.027860,0,AEM,43.0900,0.174893,0.091923,0.28502,44.0,3.24,AEM Aug 16 2019 44 Call,1
6,16,AES US Equity,0.021130,0,AES,17.1000,0.015687,0.085765,0.25274,17.0,1.12,AES Aug 16 2019 17 Call,-1
7,17,AFL US Equity,0.012022,0,AFL,49.2800,0.017070,0.085019,0.15699,50.0,1.87,AFL Aug 16 2019 50 Call,1
8,18,AG US Equity,0.060299,0,AG,6.2000,0.174232,0.120035,0.51965,7.0,0.95,AG Dec 20 2019 7 Call,1
9,20,AIG US Equity,0.012939,0,AIG,42.3200,0.246406,0.135222,0.25140,43.0,2.30,AIG Aug 16 2019 43 Call,1


In [43]:
from ortools.linear_solver import pywraplp
import requests
import json

# Instantiate a Glop solver, naming it SolveStigler.
solver = pywraplp.Solver('SolveStigler', pywraplp.Solver.GLOP_LINEAR_PROGRAMMING)

nassets = df.shape[0]
w = [[]] * nassets

# Objective: minimize the sum of (price-normalized) foods.
objective = solver.Objective()
for i in range(0, nassets):
  w[i] = solver.NumVar(0.0, 0.15, df['symbol'][i])
  objective.SetCoefficient(w[i], df['mu'][i])
objective.SetMaximization()

constraint0 = solver.Constraint(1, 1)
constraint1 = solver.Constraint(-solver.infinity(), 0.1)
constraint2 = solver.Constraint(-solver.infinity(), 0.25)
for i in range(0, nassets):
    constraint0.SetCoefficient(w[i], 1)
    constraint1.SetCoefficient(w[i], df['sigma'][i])
    constraint2.SetCoefficient(w[i], df['iv'][i])

status = solver.Solve()

capital = 1000
apiKey = "OFWNX9PJR###" # + (arba3a) (sifer) (awal 7arf min 3elit tom)

if status == solver.OPTIMAL:
    er = 0.0
    for i in range(0, nassets):
        if w[i].solution_value() > 0.0:
            #symbol = str(w[i]).split(' ')[0]
            #url = "https://api.tdameritrade.com/v1/marketdata/chains?apikey="+apiKey+"&symbol="+symbol+"&contractType=CALL&strikeCount=1&range=OTM&fromDate=2019-09-01"
            #resp = requests.get(url)
            #print(resp.content)
            #obj = json.loads(resp.content)
            #last_p = obj['underlyingPrice']
            #exp_dates = obj['callExpDateMap'].keys()
            #print(exp_dates)
            #strikes = obj['callExpDateMap'][list(exp_dates)[0]].keys()
            #print(strikes)
            #option_p = obj['callExpDateMap'][list(exp_dates)[0]][list(strikes)[0]][0]['last']
            er += w[i].solution_value()*df['mu'][i]
            print("%s [%s] @ %.2f = %0.4f with E[%.2f] # %.2f %s" % (w[i], df['position'][i], df['last_price'][i], w[i].solution_value(), df['mu'][i], capital*w[i].solution_value(), df['option'][i]))
    print("expected returns =", er)

ALLY [1] @ 27.01 = 0.1500 with E[0.17] # 150.00 ALLY Sep 20 2019 28 Call
BBBY [-1] @ 16.85 = 0.0132 with E[0.21] # 13.23 BBBY Aug 16 2019 17.5 Call
BX [1] @ 33.45 = 0.0466 with E[0.25] # 46.59 BX Sep 20 2019 34 Call
ENB [1] @ 35.82 = 0.1500 with E[0.20] # 150.00 ENB Jan 17 2020 37.5 Call
EPD [1] @ 28.30 = 0.1500 with E[0.19] # 150.00 EPD Sep 20 2019 29 Call
ET [1] @ 15.00 = 0.1500 with E[0.40] # 150.00 ET Jan 17 2020 15 Call
GSK [1] @ 41.11 = 0.1500 with E[0.13] # 150.00 GSK Aug 16 2019 42 Call
LEAF [1] @ 8.27 = 0.1500 with E[0.72] # 150.00 LEAF Aug 16 2019 9 Call
LKQ [1] @ 27.15 = 0.0402 with E[0.52] # 40.18 LKQ Aug 16 2019 27.5 Call
expected returns = 0.30858257165506403


In [34]:
nassets

480